In [ ]:
# BERT
!pip install transformers==4.15.0 sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 7.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.7 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 80.5 MB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
     |████████████████████████████████| 880 kB 60.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=45c87eeeffb4840f6ae20e45562b4cbcd5c21cc102574317dcaa591195749765
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification, AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD
import pickle

# Read CSV Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change the input directory to your own preferences
df = pickle.load(open('drive/MyDrive/AIBuilders/tagging_ds.pkl', 'rb'))
df.head()

,text,labels
0,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
1,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
2,"{'input_ids': [[tensor(5), tensor(984), tensor...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
3,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
4,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."


In [ ]:
df.iloc[0]['labels']

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    1,    1,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -1

# Initialize Tokenizer

In [ ]:
# change the input directory to your own preferences
tokenizer_th = pickle.load(open('drive/MyDrive/AIBuilders/tokenizer.pkl', 'rb'))

# Create Dataset Class

In [ ]:
class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):
        self.texts = [df.iloc[i]['text'] for i in range(df.shape[0])]
        self.labels = [df.iloc[i]['labels'] for i in range(df.shape[0])]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels


# Split Data

In [ ]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [ ]:
asdf = DataSequence(df_train)
asdf.__getitem__(1)

({'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Build Model

In [ ]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer_th))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [ ]:
model = BertModel()

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.7.attention.self.query.bias', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.encoder.layer.4.attention.self.key.bias', 'roberta.encoder.layer.3.attention.output.dense.bias', 'roberta.encoder.layer.3.output.LayerNorm.bias', 'roberta.encoder.layer.5.attention.self.key.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.7.attention.output.dense.bias', 'roberta.encoder.layer.8.intermediate.dense.weight', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.10.intermediate.dense.bias', 'robert

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
model

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

# Model Training

In [ ]:
BATCH_SIZE = 32

def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000
    
    for epoch_num in range(EPOCHS):
        total_acc_train = 0
        total_loss_train = 0

        model.train()
        
        for train_data, train_label in tqdm(train_dataloader):
            train_label = train_label[0].to(device)
            mask = train_data['attention_mask'][0].to(device)
            input_id = train_data['input_ids'][0].to(device)
            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            logits_clean = logits[0][train_label != -100]
            label_clean = train_label[train_label != -100]
            predictions = logits_clean.argmax(dim=1)
            pred_len = len(predictions)
            for i in range(pred_len):
                if predictions[i] == 0:
                    predictions[i] = -100
            
            numer = 0
            denom = 0
            for i in range(pred_len):
                if label_clean[i] == 1:
                    denom += 1
                    if predictions[i] == 1:
                        numer += 1
                elif label_clean[i] == 0 and predictions[i] == 1:
                    denom += 1
        
            acc = float(numer)/float(denom)
            # print(f"train acc: {acc}")
            total_acc_train += acc * BATCH_SIZE
            total_loss_train += loss.item() * BATCH_SIZE

            loss.backward()
            optimizer.step()
        
        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label[0].to(device)
            mask = val_data['attention_mask'][0].to(device)

            input_id = val_data['input_ids'][0].to(device)

            loss, logits = model(input_id, mask, val_label)

            logits_clean = logits[0][val_label != -100]
            label_clean = val_label[val_label != -100]
            predictions = logits_clean.argmax(dim=1)   
            pred_len = len(predictions)
            for i in range(pred_len):
                if predictions[i] == 0:
                    predictions[i] = -100       

            numer = 0
            denom = 0
            for i in range(pred_len):
                if label_clean[i] == 1:
                    denom += 1
                    if predictions[i] == 1:
                        numer += 1
                elif label_clean[i] == 0 and predictions[i] == 1:
                    denom += 1
        
            acc = float(numer)/float(denom)
            # print(f"valid acc: {acc}")
            total_acc_val += acc * BATCH_SIZE
            total_loss_val += loss.item() * BATCH_SIZE

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 1e-2
EPOCHS = 400



In [ ]:
train_loop(model, df_train, df_val)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 25/25 [00:04<00:00,  5.99it/s]


Epochs: 1 | Loss:  0.467 | Accuracy:  0.004 | Val_Loss:  0.859 | Accuracy:  0.000


100%|██████████| 25/25 [00:03<00:00,  6.43it/s]


Epochs: 2 | Loss:  0.317 | Accuracy:  0.007 | Val_Loss:  0.479 | Accuracy:  0.000


100%|██████████| 25/25 [00:03<00:00,  6.42it/s]


...

Epochs: 398 | Loss:  0.071 | Accuracy:  0.683 | Val_Loss:  0.339 | Accuracy:  0.363


100%|██████████| 25/25 [00:04<00:00,  6.08it/s]


Epochs: 399 | Loss:  0.061 | Accuracy:  0.590 | Val_Loss:  0.373 | Accuracy:  0.366


100%|██████████| 25/25 [00:04<00:00,  6.04it/s]


Epochs: 400 | Loss:  0.078 | Accuracy:  0.726 | Val_Loss:  0.356 | Accuracy:  0.382


# Evaluate Model

In [ ]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    cou = 0
    for test_data, test_label in test_dataloader:
        if cou == 20:
            break
        cou+=1

        test_label = test_label[0].to(device)
        mask = test_data['attention_mask'][0].to(device)
        input_id = test_data['input_ids'][0].to(device)
          
        loss, logits = model(input_id, mask, test_label.long())
        # print(test_data)
        # print(test_label)
        # print(loss)
        # print(logits)

        logits_clean = logits[0][test_label != -100]
        label_clean = test_label[test_label != -100]
        predictions = logits_clean.argmax(dim=1)
        pred_len = len(predictions)
        for i in range(pred_len):
            if predictions[i] == 0:
                predictions[i] = -100
        # print(logits_clean)
        # print(label_clean)
        # print(predictions)

  
        a = tokenizer_th.convert_ids_to_tokens(input_id.squeeze())
        print(f"input: {''.join(a)}")
        # print(label_clean)
        # print(len(input_id[0]), len(label_clean))
        # print("________________________")

        ids_to_labels = {0:'f', 1:'i', -100:'f'}
        prediction_label = [ids_to_labels[i] for i in predictions.tolist()]
        print("PREDICTIONS")
        for i in range(len(predictions)):
          if prediction_label[i] == 'i':
            print(a[i])
        print("LABELS")
        for i in range(len(label_clean)):
          if label_clean[i] == 1:
            print(a[i])   

        # acc = (predictions == label_clean).float().mean()

        numer = 0
        denom = 0
        for i in range(pred_len):
            if label_clean[i] == 1:
                denom += 1
                if predictions[i] == 1:
                    numer += 1
            elif label_clean[i] == 0 and predictions[i] == 1:
                denom += 1
        
        acc = float(numer)/float(denom)
        print(f"ACC: {acc}")
        print("-----------------------------------")
        total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    # print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')
    print(f'Test Accuracy: {total_acc_test / cou: .3f}')


evaluate(model, df_test)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


input: <s>▁แต่ก่อนสนใจการเมืองเพราะเพื่อนพ่อเป็นสสเวลาฟัวดิเบตได้ความรู้แนวคิดคําคมแต่หลังจากไม่มีการเลือกตั้งไม่ได้ติดตามห่างหายจนปีนี้มีรู้สึกตื่นเต้นที่จะได้ฟังแง่คิดแต่ละคนมุมมอง#พรรคอนาคตใหม่คนรุ่นใหม่อยากหนีจากวงจรอุบาทของการเมือง▁อยากเห็น▁ประเทศไทยเดินหน้ามีกินมีใช้▁#ไทยรัฐเลือกตั้ง62</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

# Predict One Sentence

In [ ]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer_th(texts, padding='max_length', max_length=512, truncation=True)
    c = tokenizer_th.convert_ids_to_tokens(tokenized_inputs.input_ids)
    print(f"ids_to_tokens: {c}")

    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        else:
            try:
              label_ids.append(2)
            except:
                label_ids.append(-100)

        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer_th(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'][0].unsqueeze(0).to(device)
   
    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)
   
    logits = model(input_id, mask, None)
    # print(input_id)
    # print(mask)
    # print(label_ids)
    logits_clean = logits[0][label_ids != -100]
    # print(logits_clean)
    # print(len(logits[0][0]), len(logits_clean))

    predictions = logits_clean.argmax(dim=1).tolist()
    ids_to_labels = {0:'f', 1:'i'}
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(prediction_label)

    tokenized_inputs = tokenizer_th(sentence, padding='max_length', max_length=512, truncation=True)
    c = tokenizer_th.convert_ids_to_tokens(tokenized_inputs.input_ids)
    print(c)
    for i, j in enumerate(prediction_label):
        if j == 'i':
            print(c[i], c[i+1], c[i+2])
            
evaluate_one_text(model, 'ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ')

ids_to_tokens: ['<s>', '▁', 'ประเทศ', 'เรา', 'ผลิต', 'และ', 'ส่งออก', 'ยา', '▁', 'สูบ', 'เยอะ', 'สุดในโลก', 'จิง', 'ป่าว', 'คับ', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

In [ ]:
FILE = "tagging.pth"
torch.save(model.state_dict(), FILE)